# 1. Main data wrangling

Explore later: (https://github.com/dunovank/jupyter-themes)


In [1]:
import pandas as pd
import os
from datetime import datetime
from datetime import timedelta
import numpy as np
import re
import math
import json
import geopandas as gp

In [ ]:
def load_incidents(filename):
    #read csv
    csv_path = os.path.join("data/traffic_incidents", filename)
    df = pd.read_csv(csv_path)
    #drop nan values, plus useless columns
    df = df.dropna()
    df = df.drop(["año_cierre", "mes_cierre", "mes"], axis=1)
    
    #drop badly recorded data
    miss = []
    r = re.compile('.{2}:.{2}:.{2}')
    for i, row in df.iterrows():
        if (r.match(row["hora_creacion"])) and (r.match(row["hora_cierre"])):
            continue
        else:
            miss.append(i)
        
    df = df.drop(miss)
    
    #renaming columns
    df = df.rename(columns={
        "fecha_creacion" : "start_date",
        "hora_creacion" : "start_hour",
        "dia_semana" : "weekday",
        "codigo_cierre" : "incident_code",
        "fecha_cierre" : "end_date",
        "hora_cierre" : "end_hour",
        "delegacion_inicio" : "start_county",
        "incidente_c4" : "type_incident",
        "latitud" : "lat",
        "longitud" : "lng",
        "clas_con_f_alarma" : "class_incident",
        "tipo_entrada" : "aid_channel",
        "delegacion_cierre" : "end_county" 
    })
    #renaming county values
    for col in ["start_county", "end_county"]:
        df = df.replace({col: {"GUSTAVO A. MADERO": "GAM",
                               "ALVARO OBREGON": "AOB",
                              "CUAJIMALPA" : "CUA",
                              "IZTAPALAPA" : "IZP",
                              "AZCAPOTZALCO" : "AZC",
                              "TLALPAN" : "TLA",
                              "MIGUEL HIDALGO" : "MHI",
                              "CUAUHTEMOC" : "CUH",
                              "IZTACALCO" : "IZC",
                              "VENUSTIANO CARRANZA" : "VCA",
                              "MAGDALENA CONTRERAS" : "MCO",
                              "BENITO JUAREZ" : "BJU",
                              "COYOACAN" : "COY",
                              "XOCHIMILCO" : "XOC",
                              "TLAHUAC" : "TLH",
                              "MILPA ALTA" : "MAL"}
                        })
    #renaming weekdays
    df = df.replace({"weekday":{"Lunes" : "MO",
                               "Martes" : "TU",
                               "Miércoles" : "WE",
                               "Jueves" : "TH",
                               "Viernes" : "FR",
                               "Sábado" : "SA",
                               "Domingo" : "SU"}})
    #renaming incident code
    def change_code(x):
        if "(D)" in x:
            return "duplicated"
        elif "(N)" in x:
            return "not found"
        elif "(A)" in x:
            return "affirmative"
        elif "(F)" in x:
            return "false"
        else:
            return "informative"
    
    df["incident_code"] = df["incident_code"].apply(change_code)
    
    #renaming class_incident
    df = df.replace({"class_incident":{"EMERGENCIA" : "Emergencies",
                               "URGENCIAS MEDICAS" : "Medical emergencies",
                               "FALSA ALARMA" : "False alarm",
                               "DELITO" : "Crime"}
                    })
    #renaming aid_channel
    df = df.replace({"aid_channel":{"LLAMADA DEL 911" : "911 call",
                               "LLAMADA APP911" : "911 App",
                               "BOTÓN DE AUXILIO" : "Help button",
                               "RADIO" : "Radio",
                               "ZELLO" : "Zello",
                               "REDES" : "Social media",
                               "CÁMARA" : "Camera"}
                    })
    #creating new column: time_response
    FMT = "%H:%M:%S"
    for i, row in df.iterrows():
        time1 = datetime.strptime(row["start_hour"], FMT)
        time2 = datetime.strptime(row["end_hour"], FMT)
        tdelta = (time2 - time1).total_seconds() // 60

        if tdelta < 0:
            tdelta = -tdelta
        df.at[i, "time_response"] = tdelta
      
    return df

In [ ]:
df17 = load_incidents("incidents_2017.csv")
#df18 = load_incidents("incidents_2018.csv")
#df19 = load_incidents("incidents_2019.csv")
#df20 = load_incidents("incidents_2020.csv")

In [ ]:
df20.to_csv("df20.csv")

In [ ]:
#transform start_date for df19
miss_date = []
r = re.compile('.{2}/.{2}/.{4}')
for i, row in df19.iterrows():
    if (r.match(row["start_date"])):
        continue
    else:
        date = row["start_date"].split("/")
        ndate= date[0]+ "/" + date[1] + "/" + "20" + date[2]
        df19.at[i,"start_date"] = ndate

## 1.2 Concatenate all dfs

In [ ]:
CDMX_1720 = pd.concat([df17, df18, df19, df20], ignore_index=True)

In [ ]:
CDMX_1720.info()

In [ ]:
CDMX_1720.to_csv("cdmx_c5_17to20.csv")

# 2. Incident rate table

In [ ]:
vfleet = pd.read_csv("data/vehicle_fleet/fleet_2018.csv")

In [ ]:
vfleet = vfleet.drop(["PROD_EST", "COBERTURA", "ESTATUS"], axis=1)

In [ ]:
#keep only records from CDMX
dprows = []
for i, row in vfleet.iterrows():
    if row["ID_ENTIDAD"] != 9:
        dprows.append(i)

vfleet = vfleet.drop(dprows)

In [ ]:
#get total vechicle fleet per county
sum_cols = list(vfleet)

del sum_cols[0:3]

vfleet['total_fleet'] = vfleet[sum_cols].sum(axis=1)

vfleet

In [ ]:
#renaming column values
vfleet = vfleet.replace({"ID_MUNICIPIO": {5 : "GAM",
                               10 : "AOB",
                              4 : "CUA",
                              7 : "IZP",
                              2: "AZC",
                              12 : "TLA",
                              16 : "MHI",
                              15 : "CUH",
                              6 : "IZC",
                              17 : "VCA",
                              8 : "MCO",
                              14 : "BJU",
                              3 : "COY",
                              13 : "XOC",
                              11 : "TLH",
                              9 : "MAL"}
                        })

In [ ]:
#create copy of df18 and drop duplicated reports
unique18 = df18.copy()

dup = []
for i, row in unique18.iterrows():
    if (row["incident_code"] == "duplicated") or (row["incident_code"] == "false") or (row["incident_code"] == "not found"):
        dup.append(i)

unique18 = unique18.drop(dup)

In [ ]:
#get total of incicents per county
incidents_county18 = unique18.groupby("start_county").count()["folio"]

total_incidents18 = pd.DataFrame({"total_incident": incidents_county18})

In [ ]:
rate_incident = vfleet.merge(total_incidents18, left_on="ID_MUNICIPIO", right_on="start_county")

In [ ]:
#calculate incident rate for every 10k vehicles, per county
for i,row in rate_incident.iterrows():
    rate_incident.at[i, "rate_incident"] = row["total_incident"] / (row["total_fleet"] / 10000)

In [ ]:
rate_incident

In [ ]:
rate_incident.to_csv("rate_incident_10k.csv")

# 3. Waffle charts

In [ ]:
def table_waffle(df):
    wdf = df.copy()
    
    wdf = wdf.drop(["folio", "start_hour", "weekday", "incident_code", "end_date", "end_hour", "start_county", "lat",
             "lng", "class_incident", "aid_channel", "end_county", "geopoint", "time_response"], axis=1)
    
    wdf["count"] = 1
    
    for i, row in wdf.iterrows():
        wdf.at[i,"start_date"] = row["start_date"].split("/")[2]
    
    wdf = wdf.replace({"type_incident":{"accidente-volcadura " : "Otros",
                               "accidente-ciclista" : "Otros",
                               "accidente-persona atrapada / desbarrancada" : "Otros",
                               "accidente-vehículo atrapado-varado" : "Otros",
                                   "accidente-choque con prensados" : "Otros",
                                   "cadáver-atropellado" : "Otros",
                                   "cadáver-accidente automovilístico" : "Otros",
                                   "sismo-choque con lesionados" : "Otros",
                                   "detención ciudadana-atropellado" : "Otros",
                                   "accidente-vehiculo desbarrancado" : "Otros",
                                   "accidente-otros" : "Otros",
                                   "detención ciudadana-accidente automovilístico" : "Otros",
                                   "Detención ciudadana-accidente automovilístico" : "Otros",
                                   "accidente-ferroviario" : "Otros",
                                   "accidente-monopatín" : "Otros",
                                   "sismo-persona atropellada" : "Otros",
                                   "accidente-volcadura" : "Otros",
                                   "accidente-vehiculo atrapado" : "Otros",
                                    "sismo-choque sin lesionados" : "Otros",
                                    "sismo-choque con prensados" : "Otros"}
                  })
    return wdf

In [ ]:
w17 = table_waffle(df17)
w18 = table_waffle(df18)
w19 = table_waffle(df19)
w20 = table_waffle(df20)

In [ ]:
waffle_1720 = pd.concat([w17, w18, w19, w20], ignore_index=True)

In [ ]:
wtotals = waffle_1720.groupby(["start_date", "type_incident"]).sum()

In [ ]:
wtotals.to_csv("waffle_1720.csv")

# 4. Incident rate json

In [149]:
i_rate = pd.read_csv("rate_incident_10k.csv")

In [150]:
i_rate = i_rate.replace({"ID_MUNICIPIO":{
                                        "AZC" : "Azcapotzalco",
                                        "COY" : "Coyoacán",
                                        "CUA" : "Cuajimalpa",
                                        "GAM" : "Gustavo A. Madero",
                                        "IZC" : "Iztacalco",
                                        "IZP" : "Iztapalapa",
                                        "MCO" : "Magdalena Contreras",
                                        "MAL" : "Milpa Alta",
                                        "AOB" : "Álvaro Obregón",
                                        "TLH" : "Tláhuac",
                                        "TLA" : "Tlalpan",
                                        "XOC" : "Xochimilco",
                                        "BJU" : "Benito Juárez",
                                        "MHI" : "Miguel Hidalgo",
                                        "VCA" : "Venustiano Carranza",
                                        "CUH" : "Cuauhtémoc"}
                                        })

In [151]:
#drop leftover columns
i_cols = list(i_rate)

del i_cols[3]

del i_cols[-3:]

i_rate = i_rate.drop(i_cols, axis=1)

In [152]:
#round rate incident
for i, row in i_rate.iterrows():
    round_rate = round(row["rate_incident"])
    i_rate.at[i, "rate_incident"] = round_rate

i_rate["rate_incident"] = i_rate["rate_incident"].astype(int)

In [153]:
i_rate

,ID_MUNICIPIO,total_fleet,total_incident,rate_incident
0,Azcapotzalco,311310,3709,119
1,Coyoacán,498273,5576,112
2,Cuajimalpa,157039,1240,79
3,Gustavo A. Madero,588619,8262,140
4,Iztacalco,253685,3444,136
5,Iztapalapa,773593,10474,135
6,Magdalena Contreras,132052,1012,77
7,Milpa Alta,55756,482,86
8,Álvaro Obregón,434976,5537,127
9,Tláhuac,135607,1766,130


In [154]:
i_dict = i_rate.to_dict(orient = 'records')

In [155]:
with open('rates_county18.json', 'w') as file:
    json.dump(i_dict, file)

In [148]:
#i_rate.to_dict(r'rates_county18.json', orient = 'records')

# 5. C5 and C2 locations (json)

In [ ]:
centers = pd.read_csv("data/centers/c5_c2_centers.csv")

In [ ]:
centers

In [ ]:
centers_loc = []
for i, row in centers.iterrows():
    center = {}
    center["Name"] = row["Nombre"]
    center["County"] = row["Alcaldia"]
    center["Location"] = [row["lat"], row["long"]]
    centers_loc.append(center)

In [ ]:
with open('centers_loc.json', 'w') as file:
    json.dump(centers_loc, file)

# 6. C5 cameras (json)

In [156]:
cameras = pd.read_csv("data/cameras/c5_cameras.csv")

In [157]:
cameras.head(2)

,ESQUINA,COLONIA,ALCALDIA,BOTON,ALTAVOZ,LONGITUD,LATITUD,geopoint,TIPO DE POSTE,ESTATUS CONECTIVIDAD,INCREMENTO 100 Mbps
0,ENTRE C. ROSA TREPADORA Y C. DR. S. GONZALEZ H...,TORRES DE MIXCOAC,ALVARO OBREGON,SIN BOTON,CON ALTAVOZ,-99.191936,19.371661,"19.371661,-99.191936",9m,WIFI GRATUITO_ACTIVO,SI
1,C. ROSA TREPADORA,MOLINO DE ROSAS,ALVARO OBREGON,CON BOTON,CON ALTAVOZ,-99.193498,19.372175,"19.372175,-99.193498",9m,WIFI GRATUITO_ACTIVO,SI


In [159]:
cameras.groupby(["ALCALDIA"])["TIPO DE POSTE"].count()

ALCALDIA
ALVARO OBREGON          899
AZCAPOTZALCO            707
BENITO JUAREZ           796
COYOACAN                907
CUAJIMALPA              239
CUAUHTEMOC             1448
GUSTAVO A. MADERO      1790
IZTACALCO               664
IZTAPALAPA             2100
MAGDALENA CONTRERAS     305
MIGUEL HIDALGO          926
MILPA ALTA              214
TLAHUAC                 540
TLALPAN                 753
VENUSTIANO CARRANZA     990
XOCHIMILCO              416
Name: TIPO DE POSTE, dtype: int64

In [ ]:
wtotals = waffle_1720.groupby(["start_date", "type_incident"]).sum()

In [ ]:
cameras_loc = []
for i, row in cameras.iterrows():
    camera = {}
    county = ""
    for word in row["ALCALDIA"].split():
        county += word.capitalize() + " "
    camera["County"] = county[:-1]
    camera["Location"] = [row["LATITUD "], row["LONGITUD "]]
    cameras_loc.append(camera)

In [ ]:
len(cameras_loc)

In [ ]:
with open('cameras_loc.json', 'w') as file:
    json.dump(cameras_loc, file)

# 7. Counties (geojson)

In [121]:
poligonos = gp.read_file("data/counties/counties.csv")

In [111]:
#poligonos = pd.read_csv("data/counties/counties.csv")

In [122]:
poligonos.head(3)

,NOMBRE,CLAVE_MUNICIPAL,CVE_ENTIDAD,CVEGEO,Geo Point,Geo Shape,municipio,geometry
0,Milpa Alta,009,09,09009,"19.1394565999,-99.0510954218","{""type"": ""Polygon"", ""coordinates"": [[[-98.9972...",9,None
1,Benito JuÃ¡rez,014,09,09014,"19.3806424162,-99.1611346584","{""type"": ""Polygon"", ""coordinates"": [[[-99.1367...",14,None
2,Gustavo A. Madero,005,09,09005,"19.5040652077,-99.1158642087","{""type"": ""Polygon"", ""coordinates"": [[[-99.1185...",5,None


In [113]:
type(poligonos["Geo Shape"][0])

str

In [123]:
i = 0
for x in poligonos["Geo Shape"]:
    pol = json.loads(x)
    new_coords = [[]]
    for coord in pol["coordinates"][0]:
        new_coords[0].append(coord[::-1])
    pol["coordinates"] = new_coords
    poligonos.at[i, "Geo Shape"] = json.dumps(pol) 
    i += 1

In [124]:
poligonos = poligonos.drop(["CLAVE_MUNICIPAL", "CVE_ENTIDAD", "municipio", "NOMBRE"], axis=1)

In [125]:
type(poligonos["Geo Shape"][0])

str

In [126]:
poligonos

,CVEGEO,Geo Point,Geo Shape,geometry
0,09009,"19.1394565999,-99.0510954218","{""type"": ""Polygon"", ""coordinates"": [[[19.22731...",None
1,09014,"19.3806424162,-99.1611346584","{""type"": ""Polygon"", ""coordinates"": [[[19.40155...",None
2,09005,"19.5040652077,-99.1158642087","{""type"": ""Polygon"", ""coordinates"": [[[19.58446...",None
3,09003,"19.3266672536,-99.1503763525","{""type"": ""Polygon"", ""coordinates"": [[[19.35686...",None
4,09016,"19.4280623649,-99.2045669144","{""type"": ""Polygon"", ""coordinates"": [[[19.46793...",None
5,09008,"19.2689765031,-99.2684129061","{""type"": ""Polygon"", ""coordinates"": [[[19.33609...",None
6,09011,"19.2769983772,-99.0028216137","{""type"": ""Polygon"", ""coordinates"": [[[19.32393...",None
7,09002,"19.4853286147,-99.1821069423","{""type"": ""Polygon"", ""coordinates"": [[[19.50747...",None
8,09006,"19.396911897,-99.094329797","{""type"": ""Polygon"", ""coordinates"": [[[19.42196...",None
9,09010,"19.336175562,-99.246819712","{""type"": ""Polygon"", ""coordinates"": [[[19.39474...",None


In [127]:
poligonos.set_geometry(col='Geo Shape', inplace=True)

TypeError: Input must be valid geometry objects: {"type": "Polygon", "coordinates": [[[19.22731068999911, -98.99723373999994], [19.227184459999123, -98.99744704999993], [19.226488459999103, -98.9974635999999], [19.226423519999155, -98.99706316999995], [19.226326379999104, -98.99668105999993], [19.22619039999913, -98.99642859999992], [19.226128889999114, -98.99638268999992], [19.225977169999148, -98.9962694499999], [19.2258887699991, -98.9961805999999], [19.225260769999117, -98.996362], [19.225211429999153, -98.99637625999996], [19.22513468999913, -98.99611645999993], [19.225038489999125, -98.99579087999992], [19.226242349999136, -98.99544217999993], [19.22643461999916, -98.99542281999994], [19.227514919999074, -98.9952639099999], [19.22744791999911, -98.99439442999991], [19.227426029999116, -98.99420421999996], [19.227470459999125, -98.99372541999993], [19.227278229999108, -98.99366731999997], [19.227090769999162, -98.9936106599999], [19.22693002999914, -98.9935620799999], [19.22679701

In [128]:
poligonos.to_file("counties_polygons.geojson", driver="GeoJSON", encoding='latin1')

In [48]:
poligonos = poligonos.astype({'geometry': 'str'}).dtypes

In [ ]:
with open("counties_polygons.geojson", 'w') as file:
    json.dump(poligonos, file)

# XXXXXXXXXXXXXXXXXXXXXXX

__REFERENCES__

https://pymotw.com/2/datetime/#:~:text=You%20can%20use%20datetime%20to,days%2C%20seconds%2C%20and%20microseconds.

Calculate time intervals
https://stackoverflow.com/questions/3096953/how-to-calculate-the-time-interval-between-two-time-strings

Convert time object into total minutes
https://stackoverflow.com/questions/27475583/how-to-convert-time-object-into-total-minutes-python

Assignate column values while iterating (df.at[i,"column_name"])
https://stackoverflow.com/questions/23330654/update-a-dataframe-in-pandas-while-iterating-row-by-row

Check if strings follow a format
https://stackoverflow.com/questions/14966647/check-python-string-format

Export df to json file
https://datatofish.com/export-pandas-dataframe-json/